# ALeRCE Xmatch service example

A short example for accessing the courrent data aviable on the ALeRCE servers for cross-match, using custom arbirtrary objects

In [1]:
!pip install alerce_xmatch

  Using cached https://files.pythonhosted.org/packages/96/4e/cbde3cf4b59748d4d465db8f8f769283f2a580d2abb3ba8b889fd52efe47/alerce_xmatch-0.0.2-py3-none-any.whl


For documentation visit GitHub repository: https://github.com/alercebroker/xmatch_client

Or the PyPi project: https://pypi.org/project/alerce-xmatch/

Generate data example

In [2]:
import pandas as pd

In [3]:
#input
columns = ['oid', 'ra', 'dec']
values = [
    [0, 168.821096816667, 29.6921437083333],
    [1, 164.669160169091, 16.0180675727273],
    [2, 174.65028162381, 20.726187647619],
    [3, 154.4155987, 30.8199994785714]
]
df = pd.DataFrame(values, columns=columns)

## Using Catalog class

In [4]:
from alerce_xmatch import Catalog, TargetCatalog, OutputCols
import time

Create Catalog class with test data

In [5]:
my_catalog = Catalog(df=df, name="My catalog")

## Crossmatch with Gaia (a Catalog in DataBase)

start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.GAIA)

end = time.time()

print("Catalog name: {}".format(cross_catalog.name))
print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Catalog name: My catalog
Crossmatch take: 2.014061450958252 s
Result:


,My catalog_dec,My catalog_oid,My catalog_ra,GAIA-DR2_designation,GAIA-DR2_ra,GAIA-DR2_dec,GAIA-DR2_parallax,GAIA-DR2_pmra,GAIA-DR2_pmdec,GAIA-DR2_astrometric_pseudo_colour,...,GAIA-DR2_bp_rp,GAIA-DR2_bp_g,GAIA-DR2_g_rp,GAIA-DR2_radial_velocity,GAIA-DR2_teff_val,GAIA-DR2_lum_val,GAIA-DR2_classifier_name,GAIA-DR2_best_class_name,GAIA-DR2_best_class_score,separation
0,16.018068,1.0,164.66916,Gaia DR2 3981758871818873600,164.669179,16.018048,0.107938,-3.091704,-2.052673,1.652324,...,0.572648,0.185996,0.386652,NaN,6898.6665,NaN,nTransits:2+,RRAB,0.985119,0.093911


To much info? Let's do a crossmatch with a catalog in memory (it supposed to be faster) and just see our info

In [6]:
start = time.time()

cross_catalog = my_catalog.crossmatch(TargetCatalog.ASASSN, output=OutputCols.SOURCE)  # Just my info please

end = time.time()

print("Crossmatch take: {} s".format(end - start))
print("Result:")
cross_catalog.df

Crossmatch take: 1.001159429550171 s
Result:


,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,None,0.0,168.821097,29.692144
1,None,3.0,154.415599,30.819999
2,None,1.0,164.669160,16.018068
3,None,2.0,174.650282,20.726188


I know the ra, dec position, maybe that's info is not neccesary

In [7]:
cross_catalog = my_catalog.crossmatch(TargetCatalog.ZTF, output=OutputCols.TARGET, # Just yuor info
                                     radec=False) # Don't show me the position
cross_catalog.df

,separation,ZTF_index,ZTF_oid
0,None,2186554,ZTF19aadfkyz
1,None,7331389,ZTF19aadfkza
2,None,10821724,ZTF19aaknuwq
3,None,14538933,ZTF18aabksxe
4,None,14630424,ZTF18aaanyxu
5,None,14731266,ZTF18aaxpvha
6,None,14738882,ZTF17aabtute
7,None,14738883,ZTF17aaajmde
8,None,15137915,ZTF18acbwbhp


## Using just the DataFrame
If you don't want to use that annoying Catalog class, use the crossmatch static method instead

In [8]:
from alerce_xmatch import crossmatch

In [9]:
crossmatch(df, TargetCatalog.ZTF) # df was declared on cell 3

,ZTF_index,ZTF_oid,ZTF_ra,ZTF_dec,ra,dec,separation,input_catalog_oid,input_catalog_ra,input_catalog_dec
0,2186554,ZTF19aadfkyz,154.415950,30.819799,154.415774,30.819899,None,3.0,154.415599,30.819999
1,7331389,ZTF19aadfkza,154.416161,30.819826,154.415880,30.819913,None,3.0,154.415599,30.819999
2,10821724,ZTF19aaknuwq,164.669156,16.017554,164.669158,16.017811,None,1.0,164.669160,16.018068
3,14538933,ZTF18aabksxe,168.821101,29.692145,168.821099,29.692144,None,0.0,168.821097,29.692144
4,14630424,ZTF18aaanyxu,168.821001,29.692204,168.821049,29.692174,None,0.0,168.821097,29.692144
5,14731266,ZTF18aaxpvha,164.669518,16.018026,164.669339,16.018047,None,1.0,164.669160,16.018068
6,14738882,ZTF17aabtute,164.669162,16.018068,164.669161,16.018068,None,1.0,164.669160,16.018068
7,14738883,ZTF17aaajmde,174.650281,20.726185,174.650281,20.726186,None,2.0,174.650282,20.726188
8,15137915,ZTF18acbwbhp,154.415609,30.819991,154.415604,30.819995,None,3.0,154.415599,30.819999


You can also use the crossmatch method with a Catalog object, and the output will be a Catalog as well

In [10]:
crossmatch(my_catalog, TargetCatalog.LINEAR).df # Remember print just the DataFrame

,LINEAR_id,LINEAR_class,LINEAR_period,LINEAR_A,LINEAR_mmed,LINEAR_stdev,LINEAR_rms,LINEAR_Lchi2pdf,LINEAR_nObs,LINEAR_skew,...,LINEAR_gErr,LINEAR_rErr,LINEAR_iErr,LINEAR_zErr,ra,dec,separation,My catalog_oid,My catalog_ra,My catalog_dec
0,1863652,RR Lyr ab,0.635622,0.88,16.68,0.31,0.38,1.671,250,-0.29,...,0.01,0.01,0.01,0.01,174.650267,20.726207,None,2.0,174.650282,20.726188
1,23555046,RR Lyr ab,0.514367,0.77,15.62,0.25,0.27,2.268,303,-0.66,...,0.01,0.01,0.01,0.01,164.669183,16.018061,None,1.0,164.669160,16.018068
